In [3]:
#Regex:

# for distinct 5 letter words w eaAE as second letter

regex = sorted(re.findall(r'\b(\w[aAeE]\w{3})\b',alice))
for word in sorted(set(regex)):
    print(word)

In [ ]:
# Simulations
#1) will u match ten of 20 cities right? what is prob you earn at least 2 points

countries = [1,2,3,4,5,6,7,8,9,10]
city_names=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

at_least_2 = 0
for i in range(10000):
    correct = 0
    random.shuffle(countries)
    random.shuffle(city_names)
    for i in range(10):
        country = countries[i]
        city = city_names[i]
        if country == city:
            correct += 1
            
    if correct >= 2:
        at_least_2 +=1
    
probability = at_least_2/10000
probability *100

In [ ]:
# Function

#lol, pull out lowest scores, avg

# def grading(l):

def grading(l):
    lol=[]
    for line in l:
        scores = [int(ele) for ele in line[1]]
        if len(scores) == 5:
            sorted_s= sorted(scores)
            scores = sorted_s[1:]
        else:
            sorted_s= sorted(scores)
            scores = sorted_s[2:]
        scores = scores
        avg = np.mean(scores)
        lol.append([line[0], avg])
        
    return lol


In [ ]:
# Processing arrrays

# load in
q4_name = np.load("q4a_name.npy")
q4_data = np.load("q4a_data.npy")
# ['fund_code' 'fund_type' 'years' 'alpha' 'Sharpe_d'] (115, 5)
#print correlation between number of years and data available fro each fund and funds CAPM alpha
#when answering this question, make sure to exclude any missing returns

q4_filter = ~np.isnan(q4_data[:,3])
q4_data2 = q4_data[q4_filter]

yr = q4_data2[:, q4_name == 'years'].T
alpha = q4_data2[:, q4_name == 'alpha'].T

corr = np.corrcoef(yr, alpha)
corr

#OR

# Filter out np.nan returns
q4_filter = ~np.isnan(q4_data[:,3])
q4_data2 = q4_data[q4_filter]
q4_data.shape, q4_data2.shape
# Calculate correlation coefficient
corr = np.corrcoef(q4_data2[:,(q4_name=='years')+(q4_name=='alpha')].T)[0,1]
print(f"The correlation between alpha and years of data is {corr:6.4f}")

# Calculate and print the average number of years separately for each of the five fund_types. You should not exclude funds with missing returns when performing this calculation.

q4_type = q4_data[:,1]

for num in sorted(set(q4_type)):
    num_y = np.mean(q4_data[:,2][q4_type==num])
    print(f"For type {num:1.0f}, the average number of years of data is {num_y:4.2f}.")
    
#Calculate and print the average alpha when Sharpe_d = 1 and again when Sharpe_d = 0. Please limit this calculation to the subset of funds for which years = 20. If there are any funds with mising returns, you should exclude them from the calcuation.
q4_year     = q4_data[:,2]
q4_sharpe_d = q4_data[:,4] 

r_s1 = np.mean(q4_data[(q4_year==20)*(q4_sharpe_d==1)][:,3])
r_s0 = np.mean(q4_data[(q4_year==20)*(q4_sharpe_d==0)][:,3])
print(f"Average alpha is {r_s1:7.4f} when Sharpe_d==1.")
print(f"Average alpha is {r_s0:7.4f} when Sharpe_d==0.")



In [2]:
# Simulating correlated returns
# Large-cap stock index has mean 10.1%, std of 20.2%
# Small-cap stock index has mean 12.5%, std of 32.3%

# Goal: Simulate correlated returns for lc and sc stocks
#       1. Assume the correlation coefficient is 0.00 -> cov = 0
#       2. Assume the correlation coefficient is 0.75

In [ ]:
mean   = [0.101, 0.125]   # mean returns
var_lc = 0.202**2
var_sc = 0.323**2

In [ ]:
cov    = [[var_lc, 0], [0, var_sc]]

In [ ]:
np.random.multivariate_normal(mean,cov, 10000).shape  
# -> 10000 rows and 2 columns

np.random.multivariate_normal(mean,cov, 10000).T.shape
# -> 2 rows and 10000 columns

x1, y1 = np.random.multivariate_normal(mean,cov, 10000).T
x1.shape, y1.shape

np.corrcoef(x1,y1)

# x1, y1 are 10000 pairs of uncorrelated stock returns
# Calculate the covariance when the correlation coefficient is 0.75
cov_ls = (0.75)*(0.202)*(0.323)
cov    = [[var_lc, cov_ls], [cov_ls, var_sc]]
x2, y2 = np.random.multivariate_normal(mean,cov, 10000).T
np.corrcoef(x2,y2)

In [ ]:
#Regressions
#R squared means we are explaing __% of data by this regression
# We'll use this function most of the time

def reg_m(y,x):
    X = sm.add_constant(x)          # adds a column of 1s if necessary
    results = sm.OLS(y, X).fit()  # results is the object containing regression estimates
    return results

reg_out = reg_m(y,x)
print(reg_out.summary(xname=['constant','x']))
reg_out.pvalues
reg_out.params  # estimated parameters
reg_out.tvalues[0], reg_out.tvalues[1]

# Multivariate regressions
# 1: simulate data
np.random.seed(100)

x2 = np.random.normal(0,2,(1000,1))
w2 = np.random.normal(0,2,(1000,1))
e2 = np.random.normal(0,10,(1000,1))
#2 horizontally stack
X2 = np.hstack((x2,w2))
reg_m(y2,X2).summary(xname=['constant','x2','w2'])

In [ ]:
# Estimating factor models with monthly returns
# Goal: Extract the array of years
yr     = m[:, m_names=='year'].reshape(36,)   # year filter
rf     = m[:, m_names=='rf'].reshape(36,1)    # column of risk-free returns
y      = m[:, m_names=='mret'].reshape(36,1)  # column of AAPL monthly returns
mktrf  = m[:, m_names=='mktrf'].reshape(36,1) # column of CAPM market factor returns
y_rf   = y - rf    # removes the risk-free rate from AAPL's monthly returns
reg_m(y_rf,mktrf).summary(xname=['CAPM alpha','CAPM beta'])

#This gives your CAPM alpha of 0.0039, what does this mean?
# beta of 1.2 tells your AAPL is riskier than a typical firm in the mkt
# Due to confidence interval, alpha might not be positive so it doesn't say much
# yes beta is above 1 but may not be

# if did this again with a year filter for two years:
year_filter = (yr==2016) | (yr==2017)   # | means or  and    + means or
reg_m(y_rf[year_filter],mktrf[year_filter]).summary(xname=['CAPM alpha','CAPM beta'])



In [ ]:
# Estimate 3-factor alpha and betas
smb = m[:,m_names=='smb'].reshape(36,1)
hml = m[:,m_names=='hml'].reshape(36,1)
X   = np.hstack((mktrf,smb,hml))

reg_m(y_rf,X).summary(xname=['FF alpha','FF beta','SMB','HML'])

In [ ]:
# Goal: Estimate a separate CAPM alpha for each calendar year

for year in sorted(set(yr)):
    reg_out = reg_m(y_rf[yr==year],mktrf[yr==year])
    print(f'{year} -- {reg_out.params[0]:7.4f} -- {reg_out.pvalues[0]:7.4f}')

# 0 is alpha, p-value

In [ ]:
# How would we test whether Apple's CAPM alpha in Q4 is statistically significantly
# different than its alpha in Q1-Q3?

# Standard pooled regression
# Q1 -> [y] = [1 x]
# Q2 -> [y] = [1 x]
# Q3 -> [y] = [1 x]
# Q4 -> [y] = [1 x]

# Allow return in Q4 to differ from other returns
# Q1 -> [y] = [1 0 x]
# Q2 -> [y] = [1 0 x]
# Q3 -> [y] = [1 0 x]
# Q4 -> [y] = [1 1 x]

mm = [str(element)[4:6] for element in m[:,0]]
q4 = [1 if element in ['10','11','12'] else 0 for element in mm]   # list comp version of np.where
q4 = np.array(q4).reshape(36,1)
X = np.hstack((q4,mktrf))

In [ ]:
#Omitted Variable Bias -- have a statistical model and missing a variable you care about
# this missing variable is correlated to the variables you include

np.random.seed(100)

#random variables
x3 = np.random.normal(0,2,(1000,1))
#make them correlated
w3 = x3 + np.random.normal(0,2,(1000,1))
e3 = np.random.normal(0,10,(1000,1))


y3 = 2 + 2*x3 + 3*w3 + e3
X3 = np.hstack((x3,w3))
reg_m(y3,X3).summary(xname=['constant','x3','w3']) #this includes all three and gets right variables

# BUT what if you do # Omitted variable bias when excluding w3
reg_m(y3,x3).summary(xname=['constant','x3'])
# causes overestimate

#If you are just being asked about return, don't need to subtract rf, when doing a Factor return always need to subtract the rf
# trying to explain apples excess return vs markets excess return over rf

In [ ]:
# Factor models with daily returns
ff_names = np.load('ff_names.npy')
ff_dates = np.load('ff_dates.npy')
ff_rets  = np.load('ff_rets.npy')

ff_mktrf = ff_rets[:,ff_names=='mktrf']
ff_smb   = ff_rets[:,ff_names=='smb']
ff_hml   = ff_rets[:,ff_names=='hml']
ff_rf    = ff_rets[:,ff_names=='rf']

FF = np.hstack((ff_mktrf,ff_smb,ff_hml))

aapl_drets = np.load('aapl_dret.npy')
aapl_dates = np.load('aapl_dates.npy')
tsla_drets = np.load('tsla_dret.npy')
tsla_dates = np.load('tsla_dates.npy')

np.corrcoef(aapl_drets.T,tsla_drets.T)


# Goal: Calculate correlation matrix for AAPL, TSLA, and MKTRF
X = np.hstack((aapl_drets,tsla_drets,ff_mktrf))
np.corrcoef(X.T)
# Estimate some factor models
tsla_rf = tsla_drets - ff_rf
aapl_rf = aapl_drets - ff_rf

# Estimate CAPM alpha and beta for AAPL using all dates
reg_m(aapl_rf,ff_mktrf).summary(xname=['CAPM alpha','CAPM beta'])

# Tsla's CAPM alpha and beta
reg_m(tsla_rf,ff_mktrf).summary(xname=['CAPM alpha','CAPM beta'])
# Goal: Estimate F-F 3-factor model for TSLA
reg_m(tsla_rf,FF).summary(xname=['FF3 alpha','FF3 beta','SMB','HML'])



#For pooled regression, stack returns for tsla and aapl and stack FF returns
# ASIDE ON REGRESSION SPECIFICATIONS

# Single regressions:   
# [y_aapl] = [1 FF]

# Pooled regression:
# [y_aapl] = [1 FF]
# [y_tsla] = [1 FF]

y_pooled = np.vstack((aapl_rf,tsla_rf))
FF_pooled = np.vstack((FF,FF))
# Pooled 3-Factor model for AAPL and TSLA ; this is assuming that aapl and tsla have same alpha and factor loading
reg_m(y_pooled,FF_pooled).summary(xname=['FF3 alpha','FF3 beta','SMB','HML'])

# How would we estimate a separate CAPM alpha for AAPL each calendar year?
year = np.array([element[0:4] for element in aapl_dates])
year = year.reshape(-1)   # this removes the structure
for y in sorted(set(year)):
    reg_out = reg_m(aapl_rf[year==y],ff_mktrf[year==y])
    print(f'{y}   {reg_out.params[0]:7.4f}    ({reg_out.pvalues[0]:7.4f})')
    
# How does Apple's CAPM beta vary over the 8 calendar years?
for y in sorted(set(year)):
    reg_out = reg_m(aapl_rf[year==y],ff_mktrf[year==y])
    print(f'{y}   {reg_out.params[1]:7.4f}    ({reg_out.pvalues[1]:7.4f})')
    
# Does AAPL's CAPM beta differ from TSLA's CAPM beta?
# # Testing for differences...

# POOLED
# [y_aapl] = [1 mktrf]
# [y_tsla] = [1 mktrf]

# ALLOWING FOR DIFFERENCES
# [y_aapl] = [1 0 mktrf     0]
# [y_tsla] = [1 1 mktrf mktrf]

aapl_stack = np.hstack((np.zeros(ff_mktrf.shape),ff_mktrf,np.zeros(ff_mktrf.shape)))
tsla_stack = np.hstack((np.ones(ff_mktrf.shape), ff_mktrf,ff_mktrf))
FF_stack = np.vstack((aapl_stack,tsla_stack))

reg_m(y_pooled,FF_stack).summary(xname=['AAPL CAPM alpha','TSLA-AAPL CAPM alpha',
                                        'AAPL CAPM beta', 'TSLA-AAPL CAPM beta'])

# Estimate separate Q1, Q2, Q3, Q4 alphas in a pooled regression with a single CAPM beta
qq = np.array([element[4:6] for element in ff_dates])
q1 = np.where((qq=='01')+(qq=='02')+(qq=='03'),1,0).reshape(ff_mktrf.shape)
q2 = np.where((qq=='04')+(qq=='05')+(qq=='06'),1,0).reshape(ff_mktrf.shape)
q3 = np.where((qq=='07')+(qq=='08')+(qq=='09'),1,0).reshape(ff_mktrf.shape)
q4 = np.where((qq=='10')+(qq=='11')+(qq=='12'),1,0).reshape(ff_mktrf.shape)
QQ = np.hstack((q1,q2,q3,q4,ff_mktrf))
reg_m(aapl_rf,QQ).summary()
sm.OLS(aapl_rf,QQ).fit().summary(xname=['Q1','Q2','Q3','Q4','beta'])

# How would we test whether AAPL's alpha and/or beta are different after 20191231?

# [aapl_rf] = [1 0 mktrf 0]     <= dates through 20191231
# [aapl_rf] = [1 1 mktrf mktrf] <= dates after 20191231

y2020 = np.where(years>='2020',1,0)
mktrf2020 = ff_mktrf*y2020
y2020 = np.where(years>='2020',1,0).reshape(ff_mktrf.shape)
mktrf2020 = ff_mktrf*y2020

YY = np.hstack((y2020,ff_mktrf,mktrf2020))
reg_m(aapl_rf,YY).summary(xname=['AAPL alpha before 2020','AAPL alpha after - before',
                                'AAPL beta before 2020','AAPL beta after - before'])




In [ ]:
#Is Apple's alpha statistically significantly higher or lower in Q4 than in the other quarters?
mm = [str(element)[4:6] for element in m[:,0]]
set(mm)
q4 = [1 if element in ['10','11','12'] else 0 for element in mm]
q4 = np.array(q4).reshape(36,1)
print(q4[0:12], np.mean(q4))
X2 = np.hstack((q4, mktrf, smb, hml))
print(reg_m(y, X2).summary(xname=['alpha','q4','mktrf','smb','hml']))


In [ ]:
names = np.array(['A']*100 + ['B']*100 + ['C']*100).reshape(300,1)

# Create 100 observations each for A, B, and C and stack them into a single column

a = np.random.normal(0.5,0.01,(100,1))
b = np.random.normal(1.0,0.01,(100,1))
c = np.random.normal(3.0,0.01,(100,1))

y = np.vstack((a,b,c))
y.shape
# Set of potential independent variables

x0 = np.ones((300,1))          # equals one for all 300 obs
x1 = np.where(names=='A',1,0)  # equals one for Stock A, zero otherwise
x2 = np.where(names=='B',1,0)  # equals one for Stock B, zero otherwise
x3 = np.where(names=='C',1,0)  # equals one for Stock C, zero otherwise

# For each stock, we regress y on the subset of x0 that corresponds to the stock
# Notice that we are using the stock name as a row filter

for stock in ['A','B','C']:
    # regression keeps subset of observations for single stock
    reg_out = sm.OLS(y[names==stock], x0[names==stock]).fit()
    print(f'Stock {stock}   {reg_out.params[0]:7.4f} with std error of {reg_out.bse[0]:7.4f}')

In [ ]:
# What if we want to limit the data to be between 20161001 and 20230731?

keep = (date>='20161001') * (date<='20230731')
date = date[keep]
r    = r[keep]

# This function incorporates the various steps above
# returns unstructured date filter and structured array of daily returns

def returns(ticker):
    # data is a pandas DataFrame, which is related to an array
    data = yf.download(ticker, start="2016-9-30", end="2023-10-31", progress=False)[["Adj Close"]]
    # adj is the array corresponding to the column of data of adjusted closing prices
    adj  = np.array(data[['Adj Close']])
    # calculates daily returns from adjusted closing prices
    r     = np.ones(adj.shape)
    r[0]  = np.nan
    r[1:] = adj[1:] / adj[0:-1] - 1
    # date is extracted from the index of the data DataFrame
    # data is formatted as '2016-09-26', but I slice around the dashes
    date = list(data.index.astype(str))
    date = [element[0:4]+element[5:7]+element[8:] for element in date]
    date = np.array(date).reshape(r.size)
    # create date filters and apply to both the date and return arrays
    keep = (date>='20161001') * (date<='20230731')
    date = date[keep]
    r    = r[keep]
    return (r, date)

ticker_list = ['AIZ','MTB','BDX','OKE','MET','HII','OMC','WY','SPY']
ret, dates = returns('AIZ')
# create a new array and use it to stack each of the columns of returns
# new columns are placed to the right of existing columns

rets = ret.copy()
for ticker in ticker_list[1:]:
    ret, ret_date = returns(ticker)   # Downloads data
    rets = np.hstack((rets,ret))      # Adds to rets array
    
    # ticker_list can be used as a row filter

ticker_list = np.array(ticker_list)
ticker_list

In [ ]:
# Estimate when allocation to SC is 50%, rho = 0, AUM = $1 million, cutoff = $1 million

print("Probability of a negative return when investing 50% in SC: {:5.2f}%".
      format(100*pr_loss(50, 0, 1000000, 1000000)))

def pr_loss(alloc_sc, rho, aum, cutoff):
    
    # dollar allocation to bond and small cap for given 'alloc_sc'
    alloc   = np.array([aum * (1-(alloc_sc/100)), aum * (alloc_sc/100)])  

    # covariance that gives desired correlation coef.
    c       = rho*0.031*0.323                                          
    mean    = [1.035, 1.125]                     # average **gross** returns of bonds and small cap
    cov     = [[0.031**2, c], [c, 0.323**2]]     # incorporates desired correlation in returns

    # draws 100000 pairs of bond and stock returns
    # array is (100000,2) because we simulate single
    # stock and single bond return in each iteration
    # replaces negative gross returns with zero
    
    returns = np.random.multivariate_normal(mean, cov, 100000)  
    returns[returns<0] = 0
    
    # convert returns to FVs and then sums each row
    # each element in column 0 multipled by $ allocation to bonds
    # each element in column 1 multipled by $ allocation to stocks
    # FVs for stocks and bonds are summed within each row
    value   = np.sum(returns*alloc, axis=1)
    
    return  np.mean(value<cutoff)       # returns fraction of rows with FV below cutoff

In [ ]:
winner = []

for r in np.arange(-1,1.01,0.1):
    # holding correlation coefficient constant, calculate probability
    # of negative return for allocations to SC of 0%, 1%, ..., 99%, 100%
    sim_loss  = [(a, pr_loss(a, r, 1000000,1000000)) for a in range(0,101,1)]  
    # extracts highest allocation to SC with probability of loss < 20%
    win_alloc = [tup for tup in sim_loss if tup[1]<0.20][-1]
    winner.append([r,win_alloc[0]])
    # calculates expected return based on 'winning' allocation
    e_ret = (1-win_alloc[0]/100)*3.5 + (win_alloc[0]/100)*12.5
    
    print("rho {:5.2f}, allocating {:5.2f}% to SC equity implies E[return] of {:4.2f}% and Pr(loss) of {:5.2f}%"
          .format(r,win_alloc[0], e_ret, win_alloc[1]*100))    

In-class exercises
What is AAPL's average daily return during 2020Q1?
What is the correlation in daily returns of SMB and HML?
Create a date array containing calendar years
What is the standard deviation of MKTRF during each calendar year?

In [ ]:
np.mean(aapl_drets[(aapl_dates>='20200101') * (aapl_dates<='20200331')])

np.corrcoef(ff_smb.T,ff_hml.T)

# 3. Create date array containing calendar years
years = np.array([element[0:4] for element in ff_dates])

# 4. What is the standard deviation of MKTRF during each calendar year?
np.std(ff_mktrf[years=='2020'])

for y in sorted(set(years)):
    print(y, np.std(ff_mktrf[years==y]))

In [ ]:
# Plotting the distribution of regression coefficients from a simulation
alpha = []
beta  = []
for num in range(10000):
    x = np.random.normal(0,1,(1000,1))
    y = -1 + 2*x + np.random.normal(0,10,(1000,1))
    reg_out = reg_m(y,x)
    a, b = reg_out.params
    alpha.append(a)
    beta.append(b)

plt.hist(alpha, np.linspace(-3,1,81), color='Rebeccapurple')
plt.show()


plt.scatter(d.keys(),d.values())
plt.xlabel('Letter')
plt.ylabel('Count')
plt.show()

plt.bar(d.keys(),d.values())
plt.xlabel('Letter')
plt.ylabel('Count')
plt.show()
# Loading daily returns
returns = np.load('2.7_returns.npy')
tickers = np.load('2.7_tickers.npy')
dates   = np.load('2.7_dates.npy')

plt.scatter(returns[tickers=='BDX'],returns[tickers=='SPY'])
plt.title('Scatter of BDX versus SPY')
plt.show()
np.corrcoef(returns[tickers=='BDX'],returns[tickers=='SPY'])

# We can run a 'CAPM-style' regression
# (I am not adjusting for the level of the risk-free rate)

y = returns[tickers=='BDX'].T
x = returns[tickers=='SPY'].T

reg_out = reg_m(y,x)

plt.scatter(x,y,color='b')
plt.plot(x, reg_out.params[0] + reg_out.params[1]*x, color='r')
plt.show()

print(f'Regression line is {reg_out.params[0]:.4f} + {reg_out.params[1]:.4f} * x')

spy = returns[tickers=='SPY'].reshape(-1) # 'list' of daily returns
met = returns[tickers=='MET'].reshape(-1) # 'list' of daily returns

plt.plot(met)
plt.plot(spy,alpha=0.5,c='r')
plt.show()

np.std(met), np.std(spy)

In [ ]:
# Farma French 3-factor model for Stock A
a_rf = stock_ret[:,stock_name=='A'] - ff_ret[:,ff_name=='rf']
#[0:3] takes mktrf, smb, hml
reg_m(a_rf,ff_ret[:,0:3]).summary(xname=['alpha','mktrf beta','smb beta','hml beta'])

In [ ]:
Constant: Daily return is +2 basis points above what is predicted by CAPM beta, but the difference is statistically insignificant (p-value of 0.703).
Mktrf: Market beta is estimated to be around 0.53; because the 95 percent confidence interval is between 0.47 and 0.59, we are highly confident that the market risk is below average.
SMB: Estimated coefficient is negative and statistically significant from zero, suggesting that the stock is a large cap stock.
HML: Estimated coefficient is negative and statistically significant from zero, suggesting that the stock is a growth stock. That said, the estimated coefficient and the upper end of the 95th percent confidence interval are both close to zero.

Use an OLS regression to test whether the average return for stocks B and C in 2020 is statistically significantly different than the average return for stocks A and D in 2020. Extract the relevant parameter and p-value from the output of reg_m() and use them to print a statement that explains whether the estimated difference is statistically significant from zero at the 5-percent level.

In [ ]:
a = stock_ret[(stock_date>="20200101")*(stock_date<="20201231"),:][:,stock_name=='A']
b = stock_ret[(stock_date>="20200101")*(stock_date<="20201231"),:][:,stock_name=='B']
c = stock_ret[(stock_date>="20200101")*(stock_date<="20201231"),:][:,stock_name=='C']
d = stock_ret[(stock_date>="20200101")*(stock_date<="20201231"),:][:,stock_name=='D']

In [ ]:
r = np.vstack((a,b,c,d))
x = np.vstack((np.ones((rows,1)),np.zeros((rows,1)),np.zeros((rows,1)),np.ones((rows,1))))
# here u are putting ones in for A & D
#[A] = [1 1]
#[B] = [1 0]
#[C] = [1 0]
#[D] = [1 1]
# The column of 1s measures the average return on B&C; this is the constant term. The 1/0 column measures the difference between the average return on stocks A&D and the average return on stocks B&C. We use the p-value on this coefficient to test whether the difference is statistically significant
r.shape, x.shape, np.mean(x)

In [ ]:
reg_out = reg_m(r,x)

In [ ]:
reg_out.summary(xname=["(Ave B and C)","(Ave A and D) - (Ave B and C)"])

In [ ]:
if reg_out.pvalues[1]<= 0.05:
    print(f"Yes, stocks A and D do have statistically significantly different returns. The coefficient\n\
    on the difference of {reg_out.params[1]:7.4f} is statistically significant at the 5-percent level\n\
    (p-value of {reg_out.pvalues[1]:6.4f}).") 
else:
    print(f"No, stocks A and D do have statistically significantly different returns. The coefficient\n\
    on the difference of {reg_out.params[1]:7.4f} is not statistically significant at the 5-percent level\n\
    (p-value of {reg_out.pvalues[1]:6.4f}).") 

In [ ]:
# See where sharpe ratio > 0


returns = np.random.normal(0.1052, 0.1538, 1000000)
SR = (returns - .01) / .1538
neg_SR = np.where(SR<0,1,0)
prob_neg = np.mean(neg_SR)
print("The probability of a negative Sharpe Ratio is {:5.2f}%.".format(100*prob_neg))


In [ ]:
#Estimate CAPM model for stock A 
a_rf = rets[:,ticker=='A'] - ff_ret[:,ff_name=='rf'].reshape(1258,1)
ff_1 = ff_ret[:,ff_name=='mktrf'].reshape(1258,1)
stockA = reg_m(a_rf, ff_1)
stockA.summary(xname=['Alpha','Mktrf'])

#Estimate 3-factor model for stock A
ff_2 = ff_ret[:, ff_name!= 'rf'].reshape(1258,3)
fff = reg_m(a_rf, ff_2)
fff.summary(xname = ['Alpha','Mktrf','SMB','HML'])

- Alpha: An Alpha of ~.5% is positive and within the confidence interval. signifies that stock, it is highly statistically significant as p value is 0.
- Mktrf: MKTRF beta of -.-311 is statistically indistinguishable from 0 as p value is 0.374.
- SMB: SMB loading is well above one, with p-value of 0.000. This is a small stock.
- HML:  HML loading is well above one, with p-value of 0.000. This is a value stock.

In [ ]:
# Check if the CAPM beta for stock B is statistically
#signficantly different at 10% level in last two years (2020-21)

b_rf = rets[:,ticker=='B'] - ff_ret[:,ff_name=='rf'].reshape(1258,1)
mktrf = ff_ret[:,ff_name=='mktrf'].reshape(1258,1)
b_rf.shape, mktrf.shape

later   = np.where(date>='20200101',1,0).reshape(1258,1)
l_mktrf = later*mktrf.reshape(1258,1)

X = np.hstack((mktrf,later,l_mktrf))
X.shape

reg_out = reg_m(b_rf,X)
reg_out.summary(xname=['Alpha','Mktrf','Alpha*(2020-2021)','Mktrf*(2020-2021)'])
if reg_out.pvalues[3]<= 0.1:
    print(f"Yes, stock B's CAPM beta is statistically significantly different in the last two years. The\n\
difference of {reg_out.params[3]:7.4f} is statistically significant (p-value of {reg_out.pvalues[3]:6.4f}).") 
else:
    print(f"No, stock B's CAPM beta is not statistically significantly different in the last two years. The\n\
difference of {reg_out.params[3]:7.4f} is not statistically significant (p-value of {reg_out.pvalues[3]:6.4f}).") 


In [ ]:
# 0 MEANS COLUMNS, 1 MEANS ROWS

q4_data2 = q4_data[(q4_date>=201807)&(q4_date<=202006)]


q4b = np.sum(~np.isnan(q4_data2),axis=0)
for num in range(len(q4_name)):
    print("Stock", q4_name[num], "has", q4b[num], "nonmissing returns.")
#Sum up all columns of non missing returns

In [ ]:
# Stock A cumulative net return that is using non missing returns

q4_dataA = q4_data[q4_name=='StockA']
q4_dataA = q4_dataA[~np.isnan(q4_dataA)]
q4c = np.cumprod(1+q4_dataA)[-1] - 1
print(f'StockA has a cumulative net return of {100*q4c:4.2f}%, based on {q4_dataA.size} non-missing returns.')


In [ ]:
Calculate the cumulative net returns for each of the four stocks using the twelve monthly returns in 2021. Store the four ending values in (4,) array q4c and then print q4c. Again, there is no need to format the outpu

In [ ]:
np.cumprod(1+sim1, axis = 1) #cumulative product within each row
np.cumprod(1+sim1, axis=1)[:,-1] -1

In [ ]:
q4_data3 = q4_data[(q4_date>=202101)&(q4_date<=202112)]
q4_data3.shape
q4c = np.cumprod(1+q4_data3, axis=0) -1
q4c = q4c[-1]
print(q4c)

In [ ]:
reg_out = reg_m(y, x)

plt.plot(x, y, '*', label='original data')
plt.plot(x, reg_out.params[0] + reg_out.params[1]*x, 'r', label='fitted line')
plt.legend()
plt.show()

In [ ]:
#Create an array that can be used to identify the returns belonging to separate years and quarters (e.g., 2216Q4, 2017Q1, ..., 2023Q3). Feel free to use list comprehension, functions, etc. Name the array yyyyqq. Print sorted(set(yyyyqq)).

yyyyqq = np.array([f'{date[:4]}Q{((int(date[4:6]) - 1) // 3) + 1}' for date in dates])
print(sorted(set(yyyyqq)))

In [ ]:
#Estimate a separate CAPM alpha for NVDA using daily returns corresponding to each value of yyyyqq. Store the estimates in a list or array and use plt.scatter() to plot them; x should be dates, y should be alphas. Please use plt.title() to give your plot a title.

In [ ]:

nvda = rets[:,names=='NVDA']
ff_mktrf = ff[:,ff_names=='mktrf']
ff_rf = ff[:,ff_names=='rf']
nvda_rf = nvda - ff_rf

capm_alpha = []
for i in sorted(set(yyyyqq)):
    reg_out = reg_m(nvda_rf[yyyyqq==i], ff_mktrf[yyyyqq==i])
    capm_alpha.append(reg_out.params[0])

plt.scatter(sorted(set(yyyyqq)), capm_alpha)
plt.xticks(rotation = 90)
plt.show()

 Use the full sample of data to test whether NVDA's CAPM alpha and CAPM beta are statistically different than Apple's CAPM alpha and CAPM beta (at a 95% confidence level in a two-sided test, which are the default assumptions for the displayed confidence intervals). Add a sentence or two that interprets the results of your hypotheses tests.

You are free to structure your test so that the four variables are:

NVDA's CAPM alpha
AAPL's CAPM alpha - NVDA's CAPM alpha
NVDA's CAPM beta
AAPL's CAPM beta - NVDA's CAPM beta

In [ ]:
aapl_rf = rets[:,names=='AAPL'] - ff[:,ff_names=='rf']

stacked_y = np.vstack((nvda_rf,aapl_rf))

stacked_nvda = np.hstack((np.zeros(ff_mktrf.shape),ff_mktrf,np.zeros(ff_mktrf.shape)))
stacked_aapl = np.hstack((np.ones(ff_mktrf.shape),  ff_mktrf,ff_mktrf))
stacked_X    = np.vstack((stacked_nvda,stacked_aapl))

reg_m(stacked_y,stacked_X).summary(xname=['NVDA alpha','AAPL alpha - NVDA alpha',
                                          'NVDA beta','AAPL beta - NVDA beta'])


Interpretation: Apple's daily CAPM alpha is 8 basis points lower than Nvidia's. However, this difference is not statistically significant at conventional levels. Under the null hypothesis of no difference, the p-value is 0.211. In contrast, Apple's CAPM beta is 0.56 points lower than Nividia's CAPM beta, and this differences is highly significant (p-value rounds down to 0.000).

In [ ]:
#Regress Costco's daily return on its daily return in the previous trading date and interpret the coefficient on lagged daily returns.
cost = rets[:,names=='COST']
cost_today = cost[1:]
cost_lag   = cost[0:-1]

print("  Today   Lagged")
print("-------  -------")
for num in range(5):
    print(f'{cost_today[num][0]:7.4f}  {cost_lag[num][0]:7.4f}')
    
reg_m(cost_today,cost_lag).summary(xname=['constant','lagged return'])


Interpretation: The coefficient on lagged returns is negative and statistically significant at the five percent level (p-value of 0.027). This tells us that when Costco's daily return is positive today it is more likely to be negative tomorrow, and vice versa. However, the R-squared (which tells us the fraction of the variation in today's stock return that can be explained by the lagged stock return) is very low.


In [ ]:
# Estimate the Fama-French three-factor (FF3) model for Costco, using all of the daily returns for 2017 through 2021. Plot the excess returns on Costco against the excess returns on the market. Plot the predicted values from the FF3 model on the same graph.

In [ ]:
years = np.array([element[0:4] for element in dates])

year_filter = (years>='2017') * (years<='2021')
cost_rf = cost - ff[:,ff_names=='rf']

y = cost_rf[year_filter]
x = ff[:,ff_names=='mktrf'][year_filter]
X = ff[:,ff_names!='rf'][year_filter]

𝑃𝑟𝑒𝑑𝑖𝑐𝑡𝑒𝑑𝑡=𝛼+𝛽𝑀𝐾𝑇𝑅𝐹∗𝑀𝐾𝑇𝑅𝐹𝑡+𝛽𝑆𝑀𝐵∗𝑆𝑀𝐵𝑡+𝑏𝑒𝑡𝑎𝐻𝑀𝐿∗𝐻𝑀𝐿𝑡

reg_out = reg_m(y,X)
pred_y  = reg_out.params[0] + reg_out.params[1]*X[:,0] + reg_out.params[2]*X[:,1] + reg_out.params[3]*X[:,2]

plt.scatter(x,y, alpha=0.5)
plt.scatter(x,pred_y, c='r', alpha = 0.5)
plt.show()

Please use the actual daily returns for each stock during 2020 to determine the annual buy-and-hold <i>net</i> return on this portfolio during 2020. Along the way, you will find it useful to calculate the cumulative annual <i>gross</i> return for each stock during 2020 (stored as `crets_2020` with shape (3,)), and to create an array with the three portfolio weights above (stored as `opt_w` with shape (1,3)).

In [ ]:
years = np.array([element[0:4] for element in dates])
rets_2020  = rets[years=='2020']
crets_2020 = np.cumprod(1+rets_2020, axis=0)[-1]   # gross returns
crets_2020, crets_2020.shape, names
opt_w    = np.array([0.13,0.47,0.40]).reshape(1,3) # same order as returns above
opt_w * crets_2020    # weights times gross returns
opt_gret = np.sum(opt_w * crets_2020) - 1


In [ ]:
Constant: Daily return is +2 basis points above what is predicted by CAPM beta, but the difference is statistically insignificant (p-value of 0.703).
Mktrf: Market beta is estimated to be around 0.53; because the 95 percent confidence interval is between 0.47 and 0.59, we are highly confident that the market risk is below average.
SMB: Estimated coefficient is negative and statistically significant from zero, suggesting that the stock is a large cap stock.
HML: Estimated coefficient is negative and statistically significant from zero, suggesting that the stock is a growth stock. That said, the estimated coefficient and the upper end of the 95th percent confidence interval are both close to zero.

In [ ]:
def summary(l):
    out = []
    for line in l:
        letter, *ret = line.split(',') # the * collects remainnig elements in list
        r = [float(element) for element in ret]
        out.append([letter, sum(r)/len(r), len(r)])
    return out

In [ ]:
# For arrays, do this to have stock name be columns
# then take max of non missing down columns

q4_dataT = q4_data.T
q4a = np.nanmax(q4_dataT[(q4_date >= 201804) & (q4_date <= 202003)], axis = 0)
for num in range(len(q4_name)):
    print(f"The maximum return for {q4_name[num]} is {q4a[num]:6.4f}.")

In [ ]:
# which of 4 stocks have lowest correlation in 2020
np.corrcoef(q4_data[:,(q4_date>=202001)&(q4_date<=202012)])
q4b_tup = ('StockB','StockD')

In [ ]:
#high book-to-market value ratio (value companies) and companies with a low book-to-market value ratio (growth)
# positive hml means value, neg means growth

# positive sml means small, negative means large

 Use a regression specification to test whether StockB's daily return in 2018 is statistically significantly different (at the 5-percent level) from its daily return in 2019 and 2020. Extract the relevant parameter and p-value from the output of reg_m() and use these values to print a statement that explains whether the estimated difference in CAPM betas is statistically significant from zero at the 5-percent level

In [ ]:
#Pull data for B from 2018-2020
b = rets[(date>='20180101')*(date<='20201231'),ticker=='StockB']
#This the subset of data for 2018-2020
date2 = date[(date>='20180101')*(date<='20201231')]

In [ ]:
x1 = np.where(date2>'20181231',1,0)
# puts 1 if date is in 2019-2020
# [1 0/1] 0/1 is for if date is later or not
b  = b.reshape(len(x1),1)
x1 = x1.reshape(len(x1),1)
reg_out = reg_m(b,x1)
reg_out.summary(xname=['Constant','Constant*(2019-2020)'])
if reg_out.pvalues[1]<= 0.05:
    print(f"Yes, stock B's return in 2019-2020 is statistically significantly different in 2018.\n\
     The difference of {reg_out.params[1]:7.4f} is statistically significant with p-value of {reg_out.pvalues[1]:6.4f}.") 
else:
    print(f"No, stock B's return in 2019-2020 is statistically significantly different in 2018.\n\
     The difference of {reg_out.params[1]:7.4f} is statistically insignificant with p-value of {reg_out.pvalues[1]:6.4f}.") 
